# Loss Functions in DidgeLab

This notebook explains the **modular loss API** for evolutionary optimization of didgeridoo shapes. You combine **loss components** (e.g. frequency tuning, scale tuning, peak count) into a **CompositeTairuaLoss**. The evolution algorithm minimizes the total loss.

**Contents:**
1. CompositeTairuaLoss – how to combine components
2. Frequency tuning – align drone/toots to target notes
3. Scale tuning – pull resonances toward a musical scale
4. Peak quantity & amplitude – encourage many, strong resonances
5. Harmonic & timbre components – integer harmonics, Q-factor, shimmer, etc.

## Setup

Add the project root so we can import `didgelab`. Then import the loss classes and NumPy.

In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
from didgelab import (
    CompositeTairuaLoss,
    FrequencyTuningLoss,
    ScaleTuningLoss,
    PeakQuantityLoss,
    PeakAmplitudeLoss,
    QFactorLoss,
    ModalDensityLoss,
    IntegerHarmonicLoss,
    NearIntegerLoss,
    StretchedOddLoss,
    HighInharmonicLoss,
    HarmonicSplittingLoss,
    note_to_freq,
)

---
## 1. CompositeTairuaLoss

**CompositeTairuaLoss** runs the acoustical simulation for a shape, detects impedance peaks, and evaluates each registered **loss component** on those peaks. The **total loss** is the sum of all component losses.

**Total loss (scalar for evolution):**

$$L_{total} = \sum_{k} L_k$$

where $L_k$ are the individual component losses (e.g. $L_{freq}$, $L_{scale}$). The evolution uses only $L_{total}$; the per-component values are stored in the loss dict for analysis.

**Usage:**
- Build a composite loss with `CompositeTairuaLoss(max_error=5.0)` (max_error controls simulation frequency resolution).
- Add components with `add_component(name, component)`.
- Use `.loss(shape)` in evolution; it returns a dict with keys for each component and `"total"`.
- Set `loss.target_freqs` (array of target frequencies in Hz) if you use `init_standard_evolution` for plotting.

In [ ]:
# Example: build a loss with two components
target_freqs_hz = np.array([73.4, 146.8])  # e.g. D1, D2
target_freqs_log = np.log2(target_freqs_hz)

loss = CompositeTairuaLoss(max_error=5.0)
loss.add_component("freq", FrequencyTuningLoss(target_freqs_log, weights=[1.0, 1.0]))
loss.add_component("volume", PeakAmplitudeLoss(target_min_amplitude=0.25, weight=20.0))
loss.target_freqs = target_freqs_hz  # optional, for init_standard_evolution

# When used in evolution: result = loss.loss(shape)  -> {"freq": ..., "volume": ..., "total": ...}

---
## 2. Frequency Tuning Loss

**Purpose:** Align the first $K$ resonance peaks to specific target frequencies (e.g. drone and first overtone). Each target is matched to the nearest detected peak; the loss is the weighted sum of pitch errors in cents, normalized.

**Formula:**

$$L_{freq} = \sum_i w_i \cdot \frac{\bigl| 1200 \cdot (\log_2 f_{target,i} - \log_2 f_{peak,i}) \bigr|}{600}$$

**Symbols:**
- $ w_i $: Weight for the $i$-th target.
- $ f_{target,i} $: Desired frequency (Hz).
- $ f_{peak,i} $: Detected peak frequency closest to the target.
- 1200: Cents per octave (since $ \log_2 $ gives octaves).
- 600: Normalization (e.g. 6 semitones).

In [ ]:
# Target notes as MIDI (e.g. -31 = D1, -19 = D2) or frequencies in Hz
target_notes = [-31, -19]
target_freqs_hz = np.array([note_to_freq(n) for n in target_notes])
target_freqs_log = np.log2(target_freqs_hz)

freq_component = FrequencyTuningLoss(
    target_freqs_log=target_freqs_log,
    weights=[1.0, 1.0],  # same weight for drone and overtone
)
# Add to composite: loss.add_component("freq", freq_component)

---
## 3. Scale Tuning Loss

**Purpose:** Pull all detected resonances toward the nearest note of a given **musical scale**. Good for instruments that should "toot" in a specific key (e.g. D minor, blues scale).

**Formula:**

$$L_{scale} = w \cdot \frac{1}{N} \sum_{i=1}^{N} \min_{F \in \mathcal{F}} \bigl| 1200 \cdot (\log_2 f_{peak,i} - \log_2 F) \bigr|$$

**Symbols:**
- $ \mathcal{F} $: Set of allowed frequencies (scale notes) from base note + intervals.
- $ f_{peak,i} $: Detected resonance frequency.
- $ \min $: Distance in cents to the closest in-tune note.
- $ w $: Weight for scale adherence.

In [ ]:
# base_note: MIDI note number of scale root (e.g. 60 = C4)
# intervals: semitone steps from root, e.g. [0, 2, 4, 5, 7, 9, 11] for major
scale_component = ScaleTuningLoss(
    base_note=60,  # C4
    intervals=[0, 2, 4, 5, 7, 9, 11],  # major scale
    weight=10.0,
)
# loss.add_component("scale", scale_component)

---
## 4. Peak Quantity Loss

**Purpose:** Penalize shapes that have **too few** resonance peaks (e.g. only drone). Encourages bore geometries that support several resonances.

**Formula:**

$$L_{qty} = w \cdot \max(0,\, N_{target} - N_{actual})$$

**Symbols:**
- $ N_{target} $: Minimum desired number of peaks.
- $ N_{actual} $: Number of peaks detected.
- $ w $: Penalty per missing peak.

In [ ]:
qty_component = PeakQuantityLoss(target_count=5, weight=2.0)
# loss.add_component("peaks", qty_component)

---
## 5. Peak Amplitude Loss

**Purpose:** Penalize **low** impedance at the resonances. Higher peaks usually mean better backpressure and a stronger, more playable sound.

**Formula:**

$$L_{amp} = w \cdot \max(0,\, A_{target} - \bar{A}_{peaks})$$

**Symbols:**
- $ A_{target} $: Desired minimum average (normalized) peak amplitude.
- $ \bar{A}_{peaks} $: Mean amplitude of detected peaks (normalized by spectrum max).
- $ w $: Weight for resonance strength.

In [ ]:
amp_component = PeakAmplitudeLoss(
    target_min_amplitude=0.25,  # expect peaks at least 25% of max
    weight=20.0,
)
# loss.add_component("volume", amp_component)

---
## 6. Q-Factor Loss

**Purpose:** Control the **sharpness** of resonances. High Q = narrow, piercing peaks; low Q = broader, warmer tone. The loss penalizes deviation from a target average Q (center frequency / bandwidth at −3 dB).

**Formula:**

$$L_Q = w \cdot \left| \left( \frac{1}{N} \sum_{i=1}^{N} \frac{f_{c,i}}{\Delta f_{i,-3\text{dB}}} \right) - Q_{target} \right|$$

**Symbols:**
- $ f_{c,i} $: Center frequency of peak $i$.
- $ \Delta f_{i,-3\text{dB}} $: Bandwidth at half-power (FWHM).
- $ Q_{target} $: Desired quality factor.
- $ w $: Weight.

In [ ]:
q_component = QFactorLoss(target_q=15.0, weight=1.0)
# loss.add_component("q", q_component)

---
## 7. Modal Density (Shimmer) Loss

**Purpose:** Reward **clustering** of peaks within a cent range to create beating/shimmer effects (chorus-like tone).

**Formula:**

$$L_{shimmer} = \frac{w}{1 + \sum_k e^{-(\Delta c_k - C)^2 / \sigma^2}}$$

**Symbols:**
- $ \Delta c_k $: Cent distance between adjacent peaks.
- $ C $: Target cluster spacing (sweet spot for beating).
- $ \sigma $: Smoothing (bandwidth of reward).
- $ w $: Weight for shimmer texture.

In [ ]:
shimmer_component = ModalDensityLoss(cluster_range_cents=30.0, weight=1.0)
# loss.add_component("shimmer", shimmer_component)

---
## 8. Integer Harmonic Loss

**Purpose:** Push the spectrum toward a **perfect harmonic series** $ f_n = n \cdot f_0 $.

**Formula:**

$$L_{int} = w \cdot \frac{1}{N} \sum_{n=1}^{N} \left| 1200 \cdot \log_2 \left( \frac{f_n}{n \cdot f_0} \right) \right|$$

**Symbols:**
- $ f_n $: Frequency of the $n$-th peak.
- $ f_0 $: Fundamental (first peak).
- $ n $: Harmonic index (1, 2, 3, …).
- $ w $: Weight for harmonic purity.

In [ ]:
int_harm_component = IntegerHarmonicLoss(weight=1.0)
# loss.add_component("harmonic", int_harm_component)

---
## 9. Near-Integer (Stretched) Loss

**Purpose:** Piano-like **stretched** harmonics: $ f_n \approx n \cdot f_0 \cdot s^n $ with stretch factor $ s $.

**Formula:**

$$L_{near} = w \cdot \sum_{n=1}^{N} \left| 1200 \cdot \log_2 \left( \frac{f_n}{n \cdot f_0 \cdot s^n} \right) \right|$$

**Symbols:**
- $ s $: Stretch factor (e.g. 1.002 for slightly sharp high harmonics).
- $ f_n $, $ f_0 $, $ n $, $ w $: As above.

In [ ]:
near_component = NearIntegerLoss(stretch_factor=1.002, weight=1.0)
# loss.add_component("stretched", near_component)

---
## 10. Stretched Odd Harmonic Loss

**Purpose:** Emphasize **odd** harmonics (1st, 3rd, 5th) with slight stretching for a hollow/woody timbre.

**Formula (conceptually):**

$$L_{odd} = w \cdot \sum \left| 1200 \cdot \log_2 \left( \frac{f_{closest}}{f_0 \cdot (2n+1)_{stretched}} \right) \right|$$

Targets are $ f_0 $, $ 3.1\,f_0 $, $ 5.2\,f_0 $ (odd-like, slightly stretched).

In [ ]:
odd_component = StretchedOddLoss(weight=1.0)
# loss.add_component("odd", odd_component)

---
## 11. High Inharmonic Loss

**Purpose:** Maximize **inharmonicity**: push harmonic ratios away from integers for a metallic, dissonant timbre.

**Formula:**

$$L_{inharm} = w \cdot \left( 0.5 - \overline{\left| \frac{f_n}{f_0} - \operatorname{round}\left(\frac{f_n}{f_0}\right) \right|} \right)$$

**Symbols:**
- $ f_n/f_0 $: Ratio to fundamental.
- $ 0.5 $: Max possible deviation from an integer ratio.
- $ w $: Weight for metallic/chaotic tone.

In [ ]:
inharm_component = HighInharmonicLoss(weight=1.0)
# loss.add_component("inharm", inharm_component)

---
## 12. Harmonic Splitting Loss

**Purpose:** Encourage **splitting** of a chosen harmonic into a close pair (nodal turbulence / gritty doublet).

**Formula (binary):**

$$L_{split} = w \cdot \mathbb{1}\bigl[ N\{\text{peaks in } [f_n \pm \delta]\} < 2 \bigr]$$

**Symbols:**
- $ f_n $: Target harmonic frequency to split.
- $ \delta $: Frequency window (Hz).
- $ N $: Count of peaks in that window.
- $ \mathbb{1} $: Penalty if fewer than 2 peaks in the window.

In [ ]:
split_component = HarmonicSplittingLoss(
    harmonic_index=1,   # second peak
    split_width_hz=5.0,
    weight=2.0,
)
# loss.add_component("split", split_component)

---
## Full example: evolution with composite loss

Combine several components and run a short evolution. Requires `Nuevolution`, a genome (e.g. `GeoGenomeA`), and optionally `init_standard_evolution` for monitoring.

In [ ]:
from didgelab import Nuevolution, GeoGenomeA, init_standard_evolution
from didgelab.app import init_app

init_app("loss_example", create_output_folder=False)

target_freqs_hz = np.array([73.4, 146.8])
target_freqs_log = np.log2(target_freqs_hz)

loss = CompositeTairuaLoss(max_error=5.0)
loss.add_component("freq", FrequencyTuningLoss(target_freqs_log, [1.0, 1.0]))
loss.add_component("scale", ScaleTuningLoss(60, [0, 2, 4, 5, 7, 9, 11], weight=5.0))
loss.add_component("peaks", PeakQuantityLoss(target_count=4, weight=2.0))
loss.add_component("volume", PeakAmplitudeLoss(0.2, weight=10.0))
loss.target_freqs = target_freqs_hz

evo = Nuevolution(
    loss,
    GeoGenomeA.build(5),
    num_generations=4,
    population_size=6,
    generation_size=4,
)
init_standard_evolution(loss.target_freqs, evo)
population = evo.evolve()
print("Best loss:", population[0].loss)